In [1]:
import nltk
from nltk.tokenize import treebank
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
import math
#from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import random
from nltk.corpus import opinion_lexicon
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 

In [2]:
lemmatizer = WordNetLemmatizer() 

In [3]:
dataset = pd.read_csv('deceptive-opinion.csv')
stop_words = set(stopwords.words('english'))

In [4]:
def train_test_split(df,test_size):
    if isinstance(test_size,float):
        test_size = round(test_size * len(df))
        
    if(test_size > len(df)):
        return 0, df

    indices = df.index.tolist()
    test_indices = random.sample(population=indices,k = test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df


In [5]:
train_df = [0]*5
test_df = [0]*5
for i in range(5):
    train_df[i],test_df[i] = train_test_split(dataset,0.3)

In [6]:
tokenizer = treebank.TreebankWordTokenizer()
def getFakenessValue(review):
    senti = 0
    words = [word.lower() for word in tokenizer.tokenize(review)]
    for word in words:
        if lemmatizer.lemmatize(word) in notfake_vocab:
            senti += 1
        if lemmatizer.lemmatize(word) in fake_vocab:
            senti -= 1
    return senti

In [23]:
accuracies = []
recalls = []
precisions = []


for j in range(5):

    fake_vocab = {}
    notfake_vocab = {}
    for i in range(len(train_df[j])):
        review = list(train_df[j]['text'])[i]
        words = word_tokenize(review)
        for word in words:
            if word.lower() not in stop_words and len(word) != 1:
                if list(train_df[j]['deceptive'])[i] == "truthful":
                    if lemmatizer.lemmatize(word.lower()) not in notfake_vocab:
                        notfake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                    else:
                        notfake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                elif lemmatizer.lemmatize(word.lower()) not in fake_vocab:
                    fake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
                else:
                    fake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
    
    
    fake_vocab = sorted(fake_vocab.items(), key = 
             lambda kv:(-kv[1], kv[0]))   
    notfake_vocab =sorted(notfake_vocab.items(), key = 
                 lambda kv:(-kv[1], kv[0]))  
    finalFakeVocab = []
    finalNotFakeVocab = []
    for i in range(2000):
        finalFakeVocab.append(fake_vocab[i][0])
        finalNotFakeVocab.append(notfake_vocab[i][0])
    fake_vocab = finalFakeVocab
    notfake_vocab = finalNotFakeVocab
    
    old_len_fakevocab = len(fake_vocab)
    old_len_notfakevocab = len(notfake_vocab)
    new_len_fakevocab = 0;
    new_len_notfakevocab = 0;
    
    while(True):
        old_len_fakevocab = len(fake_vocab)
        old_len_notfakevocab = len(notfake_vocab)
        for word in notfake_vocab:
            for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                                for lemma in synset.lemmas():
                                    if lemma.name() not in notfake_vocab:
                                        notfake_vocab.append(lemma.name())



        for word in fake_vocab:
            for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                                for lemma in synset.lemmas():
                                    if lemma.name() not in fake_vocab:
                                        fake_vocab.append(lemma.name())
        
        new_len_fakevocab = len(fake_vocab);
        new_len_notfakevocab = len(notfake_vocab);
        
        if(old_len_fakevocab == new_len_fakevocab and old_len_notfakevocab == new_len_notfakevocab):
            break

    
    test_df[j]['fakescore'] = test_df[j]['text'].apply(getFakenessValue)
    
    truePositive = trueNegative = falsePositive = falseNegative = 0
    for i in range(len(test_df[j])):
        if list(test_df[j]['deceptive'])[i] == 'deceptive' and list(test_df[j]['fakescore'])[i] < 0:
            truePositive += 1
        elif list(test_df[j]['deceptive'])[i] == 'truthful' and list(test_df[j]['fakescore'])[i] > 0:
            trueNegative += 1
        elif list(test_df[j]['deceptive'])[i] == 'truthful' and list(test_df[j]['fakescore'])[i] < 0:
            falsePositive += 1
        elif list(test_df[j]['deceptive'])[i] == 'deceptive' and list(test_df[j]['fakescore'])[i] > 0 :
            falseNegative += 1

    accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
    precision = truePositive/(truePositive + falsePositive)
    recall =truePositive/(truePositive + falseNegative) 

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    


In [24]:
len(notfake_vocab),len(fake_vocab)

(30339, 30186)

In [25]:
sum(accuracies)/len(accuracies),sum(precisions)/len(precisions),sum(recalls)/len(recalls)

(0.7053990644386113, 0.700336759223777, 0.63725296777434)

In [26]:
accuracies = []
recalls = []
precisions = []


for j in range(5):

    fake_vocab = {}
    notfake_vocab = {}
    for i in range(len(train_df[j])):
        review = list(train_df[j]['text'])[i]
        words = word_tokenize(review)
        for word in words:
            if word.lower() not in stop_words and len(word) != 1:
                if list(train_df[j]['deceptive'])[i] == "truthful":
                    if lemmatizer.lemmatize(word.lower()) not in notfake_vocab:
                        notfake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                    else:
                        notfake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                elif lemmatizer.lemmatize(word.lower()) not in fake_vocab:
                    fake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
                else:
                    fake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
    
    
    fake_vocab = sorted(fake_vocab.items(), key = 
             lambda kv:(-kv[1], kv[0]))   
    notfake_vocab =sorted(notfake_vocab.items(), key = 
                 lambda kv:(-kv[1], kv[0]))  
    finalFakeVocab = []
    finalNotFakeVocab = []
    for i in range(1000):
        finalFakeVocab.append(fake_vocab[i][0])
        finalNotFakeVocab.append(notfake_vocab[i][0])
    fake_vocab = finalFakeVocab
    notfake_vocab = finalNotFakeVocab
    
    old_len_fakevocab = len(fake_vocab)
    old_len_notfakevocab = len(notfake_vocab)
    new_len_fakevocab = 0;
    new_len_notfakevocab = 0;
    
    while(True):
        old_len_fakevocab = len(fake_vocab)
        old_len_notfakevocab = len(notfake_vocab)
        for word in notfake_vocab:
            for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                                for lemma in synset.lemmas():
                                    if lemma.name() not in notfake_vocab:
                                        notfake_vocab.append(lemma.name())



        for word in fake_vocab:
            for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                                for lemma in synset.lemmas():
                                    if lemma.name() not in fake_vocab:
                                        fake_vocab.append(lemma.name())
        
        new_len_fakevocab = len(fake_vocab);
        new_len_notfakevocab = len(notfake_vocab);
        
        if(old_len_fakevocab == new_len_fakevocab and old_len_notfakevocab == new_len_notfakevocab):
            break

    
    test_df[j]['fakescore'] = test_df[j]['text'].apply(getFakenessValue)
    
    truePositive = trueNegative = falsePositive = falseNegative = 0
    for i in range(len(test_df[j])):
        if list(test_df[j]['deceptive'])[i] == 'deceptive' and list(test_df[j]['fakescore'])[i] < 0:
            truePositive += 1
        elif list(test_df[j]['deceptive'])[i] == 'truthful' and list(test_df[j]['fakescore'])[i] > 0:
            trueNegative += 1
        elif list(test_df[j]['deceptive'])[i] == 'truthful' and list(test_df[j]['fakescore'])[i] < 0:
            falsePositive += 1
        elif list(test_df[j]['deceptive'])[i] == 'deceptive' and list(test_df[j]['fakescore'])[i] > 0 :
            falseNegative += 1

    accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
    precision = truePositive/(truePositive + falsePositive)
    recall =truePositive/(truePositive + falseNegative) 

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    


In [27]:
sum(accuracies)/len(accuracies),sum(precisions)/len(precisions),sum(recalls)/len(recalls)

(0.6968231621864379, 0.6679930937935936, 0.7516682922406642)

In [28]:
accuracies = []
recalls = []
precisions = []


for j in range(5):

    fake_vocab = {}
    notfake_vocab = {}
    for i in range(len(train_df[j])):
        review = list(train_df[j]['text'])[i]
        words = word_tokenize(review)
        for word in words:
            if word.lower() not in stop_words and len(word) != 1:
                if list(train_df[j]['deceptive'])[i] == "truthful":
                    if lemmatizer.lemmatize(word.lower()) not in notfake_vocab:
                        notfake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                    else:
                        notfake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                elif lemmatizer.lemmatize(word.lower()) not in fake_vocab:
                    fake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
                else:
                    fake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
    
    
    fake_vocab = sorted(fake_vocab.items(), key = 
             lambda kv:(-kv[1], kv[0]))   
    notfake_vocab =sorted(notfake_vocab.items(), key = 
                 lambda kv:(-kv[1], kv[0]))  
    finalFakeVocab = []
    finalNotFakeVocab = []
    for i in range(800):
        finalFakeVocab.append(fake_vocab[i][0])
        finalNotFakeVocab.append(notfake_vocab[i][0])
    fake_vocab = finalFakeVocab
    notfake_vocab = finalNotFakeVocab
    
    old_len_fakevocab = len(fake_vocab)
    old_len_notfakevocab = len(notfake_vocab)
    new_len_fakevocab = 0;
    new_len_notfakevocab = 0;
    
    while(True):
        old_len_fakevocab = len(fake_vocab)
        old_len_notfakevocab = len(notfake_vocab)
        for word in notfake_vocab:
            for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                                for lemma in synset.lemmas():
                                    if lemma.name() not in notfake_vocab:
                                        notfake_vocab.append(lemma.name())



        for word in fake_vocab:
            for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                                for lemma in synset.lemmas():
                                    if lemma.name() not in fake_vocab:
                                        fake_vocab.append(lemma.name())
        
        new_len_fakevocab = len(fake_vocab);
        new_len_notfakevocab = len(notfake_vocab);
        
        if(old_len_fakevocab == new_len_fakevocab and old_len_notfakevocab == new_len_notfakevocab):
            break

    
    test_df[j]['fakescore'] = test_df[j]['text'].apply(getFakenessValue)
    
    truePositive = trueNegative = falsePositive = falseNegative = 0
    for i in range(len(test_df[j])):
        if list(test_df[j]['deceptive'])[i] == 'deceptive' and list(test_df[j]['fakescore'])[i] < 0:
            truePositive += 1
        elif list(test_df[j]['deceptive'])[i] == 'truthful' and list(test_df[j]['fakescore'])[i] > 0:
            trueNegative += 1
        elif list(test_df[j]['deceptive'])[i] == 'truthful' and list(test_df[j]['fakescore'])[i] < 0:
            falsePositive += 1
        elif list(test_df[j]['deceptive'])[i] == 'deceptive' and list(test_df[j]['fakescore'])[i] > 0 :
            falseNegative += 1

    accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
    precision = truePositive/(truePositive + falsePositive)
    recall =truePositive/(truePositive + falseNegative) 

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    


In [29]:
sum(accuracies)/len(accuracies),sum(precisions)/len(precisions),sum(recalls)/len(recalls)

(0.7023151569186739, 0.6622677823783321, 0.7794885203561002)

In [30]:
accuracies = []
recalls = []
precisions = []


for j in range(5):

    fake_vocab = {}
    notfake_vocab = {}
    for i in range(len(train_df[j])):
        review = list(train_df[j]['text'])[i]
        words = word_tokenize(review)
        for word in words:
            if word.lower() not in stop_words and len(word) != 1:
                if list(train_df[j]['deceptive'])[i] == "truthful":
                    if lemmatizer.lemmatize(word.lower()) not in notfake_vocab:
                        notfake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                    else:
                        notfake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                elif lemmatizer.lemmatize(word.lower()) not in fake_vocab:
                    fake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
                else:
                    fake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
    
    
    fake_vocab = sorted(fake_vocab.items(), key = 
             lambda kv:(-kv[1], kv[0]))   
    notfake_vocab =sorted(notfake_vocab.items(), key = 
                 lambda kv:(-kv[1], kv[0]))  
    finalFakeVocab = []
    finalNotFakeVocab = []
    for i in range(600):
        finalFakeVocab.append(fake_vocab[i][0])
        finalNotFakeVocab.append(notfake_vocab[i][0])
    fake_vocab = finalFakeVocab
    notfake_vocab = finalNotFakeVocab
    
    old_len_fakevocab = len(fake_vocab)
    old_len_notfakevocab = len(notfake_vocab)
    new_len_fakevocab = 0;
    new_len_notfakevocab = 0;
    
    while(True):
        old_len_fakevocab = len(fake_vocab)
        old_len_notfakevocab = len(notfake_vocab)
        for word in notfake_vocab:
            for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                                for lemma in synset.lemmas():
                                    if lemma.name() not in notfake_vocab:
                                        notfake_vocab.append(lemma.name())



        for word in fake_vocab:
            for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                                for lemma in synset.lemmas():
                                    if lemma.name() not in fake_vocab:
                                        fake_vocab.append(lemma.name())
        
        new_len_fakevocab = len(fake_vocab);
        new_len_notfakevocab = len(notfake_vocab);
        
        if(old_len_fakevocab == new_len_fakevocab and old_len_notfakevocab == new_len_notfakevocab):
            break

    
    test_df[j]['fakescore'] = test_df[j]['text'].apply(getFakenessValue)
    
    truePositive = trueNegative = falsePositive = falseNegative = 0
    for i in range(len(test_df[j])):
        if list(test_df[j]['deceptive'])[i] == 'deceptive' and list(test_df[j]['fakescore'])[i] < 0:
            truePositive += 1
        elif list(test_df[j]['deceptive'])[i] == 'truthful' and list(test_df[j]['fakescore'])[i] > 0:
            trueNegative += 1
        elif list(test_df[j]['deceptive'])[i] == 'truthful' and list(test_df[j]['fakescore'])[i] < 0:
            falsePositive += 1
        elif list(test_df[j]['deceptive'])[i] == 'deceptive' and list(test_df[j]['fakescore'])[i] > 0 :
            falseNegative += 1

    accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
    precision = truePositive/(truePositive + falsePositive)
    recall =truePositive/(truePositive + falseNegative) 

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    


In [31]:
accuracies = []
recalls = []
precisions = []


for j in range(5):

    fake_vocab = {}
    notfake_vocab = {}
    for i in range(len(train_df[j])):
        review = list(train_df[j]['text'])[i]
        words = word_tokenize(review)
        for word in words:
            if word.lower() not in stop_words and len(word) != 1:
                if list(train_df[j]['deceptive'])[i] == "truthful":
                    if lemmatizer.lemmatize(word.lower()) not in notfake_vocab:
                        notfake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                    else:
                        notfake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                elif lemmatizer.lemmatize(word.lower()) not in fake_vocab:
                    fake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
                else:
                    fake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
    
    
    fake_vocab = sorted(fake_vocab.items(), key = 
             lambda kv:(-kv[1], kv[0]))   
    notfake_vocab =sorted(notfake_vocab.items(), key = 
                 lambda kv:(-kv[1], kv[0]))  
    finalFakeVocab = []
    finalNotFakeVocab = []
    for i in range(200):
        finalFakeVocab.append(fake_vocab[i][0])
        finalNotFakeVocab.append(notfake_vocab[i][0])
    fake_vocab = finalFakeVocab
    notfake_vocab = finalNotFakeVocab
    
    old_len_fakevocab = len(fake_vocab)
    old_len_notfakevocab = len(notfake_vocab)
    new_len_fakevocab = 0;
    new_len_notfakevocab = 0;
    
    while(True):
        old_len_fakevocab = len(fake_vocab)
        old_len_notfakevocab = len(notfake_vocab)
        for word in notfake_vocab:
            for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                                for lemma in synset.lemmas():
                                    if lemma.name() not in notfake_vocab:
                                        notfake_vocab.append(lemma.name())



        for word in fake_vocab:
            for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                                for lemma in synset.lemmas():
                                    if lemma.name() not in fake_vocab:
                                        fake_vocab.append(lemma.name())
        
        new_len_fakevocab = len(fake_vocab);
        new_len_notfakevocab = len(notfake_vocab);
        
        if(old_len_fakevocab == new_len_fakevocab and old_len_notfakevocab == new_len_notfakevocab):
            break

    
    test_df[j]['fakescore'] = test_df[j]['text'].apply(getFakenessValue)
    
    truePositive = trueNegative = falsePositive = falseNegative = 0
    for i in range(len(test_df[j])):
        if list(test_df[j]['deceptive'])[i] == 'deceptive' and list(test_df[j]['fakescore'])[i] < 0:
            truePositive += 1
        elif list(test_df[j]['deceptive'])[i] == 'truthful' and list(test_df[j]['fakescore'])[i] > 0:
            trueNegative += 1
        elif list(test_df[j]['deceptive'])[i] == 'truthful' and list(test_df[j]['fakescore'])[i] < 0:
            falsePositive += 1
        elif list(test_df[j]['deceptive'])[i] == 'deceptive' and list(test_df[j]['fakescore'])[i] > 0 :
            falseNegative += 1

    accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
    precision = truePositive/(truePositive + falsePositive)
    recall =truePositive/(truePositive + falseNegative) 

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    


In [32]:
sum(accuracies)/len(accuracies),sum(precisions)/len(precisions),sum(recalls)/len(recalls)

(0.7082547579919092, 0.6768825858872582, 0.7545983747196504)

In [33]:
accuracies = []
recalls = []
precisions = []


for j in range(5):

    fake_vocab = {}
    notfake_vocab = {}
    for i in range(len(train_df[j])):
        review = list(train_df[j]['text'])[i]
        words = word_tokenize(review)
        for word in words:
            if word.lower() not in stop_words and len(word) != 1:
                if list(train_df[j]['deceptive'])[i] == "truthful":
                    if lemmatizer.lemmatize(word.lower()) not in notfake_vocab:
                        notfake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                    else:
                        notfake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                elif lemmatizer.lemmatize(word.lower()) not in fake_vocab:
                    fake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
                else:
                    fake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
    
    
    fake_vocab = sorted(fake_vocab.items(), key = 
             lambda kv:(-kv[1], kv[0]))   
    notfake_vocab =sorted(notfake_vocab.items(), key = 
                 lambda kv:(-kv[1], kv[0]))  
    finalFakeVocab = []
    finalNotFakeVocab = []
    for i in range(500):
        finalFakeVocab.append(fake_vocab[i][0])
        finalNotFakeVocab.append(notfake_vocab[i][0])
    fake_vocab = finalFakeVocab
    notfake_vocab = finalNotFakeVocab
    
    old_len_fakevocab = len(fake_vocab)
    old_len_notfakevocab = len(notfake_vocab)
    new_len_fakevocab = 0;
    new_len_notfakevocab = 0;
    
    while(True):
        old_len_fakevocab = len(fake_vocab)
        old_len_notfakevocab = len(notfake_vocab)
        for word in notfake_vocab:
            for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                                for lemma in synset.lemmas():
                                    if lemma.name() not in notfake_vocab:
                                        notfake_vocab.append(lemma.name())



        for word in fake_vocab:
            for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                                for lemma in synset.lemmas():
                                    if lemma.name() not in fake_vocab:
                                        fake_vocab.append(lemma.name())
        
        new_len_fakevocab = len(fake_vocab);
        new_len_notfakevocab = len(notfake_vocab);
        
        if(old_len_fakevocab == new_len_fakevocab and old_len_notfakevocab == new_len_notfakevocab):
            break

    
    test_df[j]['fakescore'] = test_df[j]['text'].apply(getFakenessValue)
    
    truePositive = trueNegative = falsePositive = falseNegative = 0
    for i in range(len(test_df[j])):
        if list(test_df[j]['deceptive'])[i] == 'deceptive' and list(test_df[j]['fakescore'])[i] < 0:
            truePositive += 1
        elif list(test_df[j]['deceptive'])[i] == 'truthful' and list(test_df[j]['fakescore'])[i] > 0:
            trueNegative += 1
        elif list(test_df[j]['deceptive'])[i] == 'truthful' and list(test_df[j]['fakescore'])[i] < 0:
            falsePositive += 1
        elif list(test_df[j]['deceptive'])[i] == 'deceptive' and list(test_df[j]['fakescore'])[i] > 0 :
            falseNegative += 1

    accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
    precision = truePositive/(truePositive + falsePositive)
    recall =truePositive/(truePositive + falseNegative) 

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    


In [34]:
sum(accuracies)/len(accuracies),sum(precisions)/len(precisions),sum(recalls)/len(recalls)

(0.6952928218432929, 0.66137457491443, 0.7918397566996694)

In [39]:
train_df = [0]*5
test_df = [0]*5
for i in range(5):
    train_df[i],test_df[i] = train_test_split(dataset,0.4)

In [40]:
accuracies = []
recalls = []
precisions = []


for j in range(5):

    fake_vocab = {}
    notfake_vocab = {}
    for i in range(len(train_df[j])):
        review = list(train_df[j]['text'])[i]
        words = word_tokenize(review)
        for word in words:
            if word.lower() not in stop_words and len(word) != 1:
                if list(train_df[j]['deceptive'])[i] == "truthful":
                    if lemmatizer.lemmatize(word.lower()) not in notfake_vocab:
                        notfake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                    else:
                        notfake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in notfake_vocab:
#                                     notfake_vocab[lemma.name()] = 1
#                                 else:
#                                     notfake_vocab[lemma.name()] += 1
                elif lemmatizer.lemmatize(word.lower()) not in fake_vocab:
                    fake_vocab[lemmatizer.lemmatize(word.lower())] = 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
                else:
                    fake_vocab[lemmatizer.lemmatize(word.lower())] += 1
#                     for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in fake_vocab:
#                                     fake_vocab[lemma.name()] = 1
#                                 else:
#                                     fake_vocab[lemma.name()] += 1
    
    
    fake_vocab = sorted(fake_vocab.items(), key = 
             lambda kv:(-kv[1], kv[0]))   
    notfake_vocab =sorted(notfake_vocab.items(), key = 
                 lambda kv:(-kv[1], kv[0]))  
    finalFakeVocab = []
    finalNotFakeVocab = []
    for i in range(500):
        finalFakeVocab.append(fake_vocab[i][0])
        finalNotFakeVocab.append(notfake_vocab[i][0])
    fake_vocab = finalFakeVocab
    notfake_vocab = finalNotFakeVocab
    
    old_len_fakevocab = len(fake_vocab)
    old_len_notfakevocab = len(notfake_vocab)
    new_len_fakevocab = 0;
    new_len_notfakevocab = 0;
    
    while(True):
        old_len_fakevocab = len(fake_vocab)
        old_len_notfakevocab = len(notfake_vocab)
        for word in notfake_vocab:
            for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                                for lemma in synset.lemmas():
                                    if lemma.name() not in notfake_vocab:
                                        notfake_vocab.append(lemma.name())



        for word in fake_vocab:
            for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
                                for lemma in synset.lemmas():
                                    if lemma.name() not in fake_vocab:
                                        fake_vocab.append(lemma.name())
        
        new_len_fakevocab = len(fake_vocab);
        new_len_notfakevocab = len(notfake_vocab);
        
        if(old_len_fakevocab == new_len_fakevocab and old_len_notfakevocab == new_len_notfakevocab):
            break

    
    test_df[j]['fakescore'] = test_df[j]['text'].apply(getFakenessValue)
    
    truePositive = trueNegative = falsePositive = falseNegative = 0
    for i in range(len(test_df[j])):
        if list(test_df[j]['deceptive'])[i] == 'deceptive' and list(test_df[j]['fakescore'])[i] < 0:
            truePositive += 1
        elif list(test_df[j]['deceptive'])[i] == 'truthful' and list(test_df[j]['fakescore'])[i] > 0:
            trueNegative += 1
        elif list(test_df[j]['deceptive'])[i] == 'truthful' and list(test_df[j]['fakescore'])[i] < 0:
            falsePositive += 1
        elif list(test_df[j]['deceptive'])[i] == 'deceptive' and list(test_df[j]['fakescore'])[i] > 0 :
            falseNegative += 1

    accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
    precision = truePositive/(truePositive + falsePositive)
    recall =truePositive/(truePositive + falseNegative) 

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    


In [42]:
sum(accuracies)/len(accuracies),sum(precisions)/len(precisions),sum(recalls)/len(recalls)

(0.6969938963477029, 0.7112625243585258, 0.6457704794706511)

[0.7028301886792453,
 0.7285067873303167,
 0.6651376146788991,
 0.6153846153846154,
 0.6333333333333333]